# Iterating through the list of authors

In [1]:
import pickle
import re
import pandas as pd
from bs4 import BeautifulSoup
from parsing_page import parsing_author_page, parsing_article_page
import time
import requests
from os.path import exists
from os import makedirs
import random 


In [2]:
from data_types import Author, AuthorsDB
from data_types import Publication, PublicationsDB
from data_types import Abstract, AbstractsDB
audb = AuthorsDB()
audb.load()
pubdb = PublicationsDB()
pubdb.load()
absdb = AbstractsDB()
absdb.load()

In [3]:
filename = "../data/mathnet_iam_authors_dict.pkl"
with open(filename,'rb') as inp:
    authors_dict = pickle.load(inp)

# Main functions for 

In [4]:
print(len(authors_dict))
for indx, item in authors_dict.items():
    # if indx <5:
    if 'Горобец' in item['name']:
        print(item)

871
{'name': 'Горобец Андрей Владимирович', 'mn_id': '27667'}


In [5]:
def need_to_scrap(author, authors_db:AuthorsDB, inlist = True):
    if not inlist:
        if authors_db.check_key(author["mn_id"]):
            # print(f"We have him {author} in authors_db")
            return False
        else:
            # print(f"We have him {author} in authors_db")
            print(f"We don't have him {author} in authors_db")
            return True        
    interesting_list = ["Бондаренко", "Поляков", "Якобовский", "Четверушкин",'Савенков', 'Горобец']
    # interesting_list = ["Бондаренко", "Якобовский"]
    # interesting_list = ["Бондаренко"]
    for person in interesting_list:
        if person in author['name']:
            # we can demand some other conditions
            # if not authors_db.check_key(author["mn_id"]):
            if authors_db.check_key(author["mn_id"]):
                print(f"We have him {author} in authors_db")
                return False
            else:
                # print(f"We have him {author} in authors_db")
                print(f"We don't have him {author} in authors_db")
                return True
    return False 

In [6]:
def save_html(data, mnid, name, status='write'):
    newpath = r'../data/'+mnid+'/'
    if not exists(newpath):
        makedirs(newpath)
    filename = "../data/"+mnid+"/"+name+".pkl"
    if status == 'write':        
        with open(filename,'wb') as outp:
            pickle.dump(data, outp, pickle.HIGHEST_PROTOCOL)

In [7]:
from fake_useragent import UserAgent
for i in range(5):
    agent = UserAgent().chrome
    print(agent)

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1309.0 Safari/537.17
Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.2309.372 Safari/537.36
Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1623.0 Safari/537.36
Mozilla/5.0 (X11; OpenBSD i386) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.124 Safari/537.36


In [8]:
def get_author_info(mnid, status="not_saving", web='offline',rand=0, agent = 'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1467.0 Safari/537.36'):
    print("../data/"+mnid+"/mn_author_"+mnid+"_page.pkl")
    print(exists("../data/"+mnid+"/mn_author_"+mnid+"_page.pkl"))
    if exists("../data/"+mnid+"/mn_author_"+mnid+"_page.pkl"):
        print("We read from file")
        filename = "../data/"+mnid+"/mn_author_"+mnid+"_page.pkl"
        with open(filename,'rb') as inp:
            author_page = pickle.load(inp)
    else:
        print(f"we need to request page for author mnid = {mnid}")
        if web=='offline':
            return None
        page_link = f"http://www.mathnet.ru/php/person.phtml?option_lang=rus&personid={mnid}"
        
        # Chancge 'User-Agent' HEADERRRRR after some time of use !!!!        
        # headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        # headers = {'User-Agent':'Mozilla/5.0 (X11; CrOS i686 4319.74.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.57 Safari/537.36'}
        # headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'}
        # headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1467.0 Safari/537.36'}
        headers = {'User-Agent':agent}
        # page_link ="http://mi.mathnet.ru/+ {}"
        time.sleep(4*(1+rand))
        response = requests.get(page_link, headers=headers, timeout=None)
        if response.status_code == 200:
            author_page = response.content
        if status=="saving":
            save_html(author_page, mnid, "mn_author_"+mnid+"_page",status='write')
        # author_page
                    
    soup = BeautifulSoup(author_page, 'html.parser')
    # TODO:
    # Need to check if this is real page and not for bot response
    return parsing_author_page(soup) 

In [9]:
def get_pub_info(mn_id, mnlink, status="not_saving",web='offline',rand=0, agent = 'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1467.0 Safari/537.36'):
    print("../data/"+mn_id+"/"+mnlink+".pkl")
    print(exists("../data/"+mn_id+"/"+mnlink+".pkl"))
    if exists("../data/"+mn_id+"/"+mnlink+".pkl"):
        print("We read from file")
        filename = "../data/"+mn_id+"/"+mnlink+".pkl"
        with open(filename,'rb') as inp:
            pub_page = pickle.load(inp)
    else:
        print("we don't find that file")
        if web == 'offline':
            return None   
        print(f"we need to request page for pub_page mnlink = {mnlink}")
        page_link = "http://mi.mathnet.ru/" + mnlink
        print(page_link)
        # Chancge 'User-Agent' HEADERRRRR after some time of use !!!!
        # headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        # headers = {'User-Agent':'Mozilla/5.0 (X11; CrOS i686 4319.74.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.57 Safari/537.36'}
        # headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'}
        # headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1467.0 Safari/537.36'}
        headers = {'User-Agent':agent}
        # 
        time.sleep(3*(1+rand))
        response = requests.get(page_link, headers=headers, timeout=None)        
        print(response.status_code)        
        if response.status_code == 200:
            pub_page = response.content            
            print("we save file: " + mnlink)
            if status=="saving":
                save_html(pub_page, mn_id, mnlink,status='write')
        else:
            print(response.status_code)            
            print("response.status_code != 200:")
            print(f"we was broken with mn_id = {mn_id} mnlink = {mnlink}")
            return "Stop"
        # pub_page        
    soup = BeautifulSoup(pub_page, 'html.parser')
    
    return parsing_article_page(soup)

In [10]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org',timeout=None)                       
    soup = BeautifulSoup(ip.content, 'html.parser')
    print(soup.find('body').text)

In [11]:
for i in range(3):
    checkIP()

Current IP Address: 89.39.104.204
Current IP Address: 89.39.104.204
Current IP Address: 89.39.104.204


we use this ips:
Day2

- 89.39.104.204 NL-FREE#88
Day1
- 89.39.104.194 NL-FREE#88
- NL-FREE#89
- 89.39.107.157 NL-FREE#90
- 93.190.140.117 NL-91
- 185.177.126.151 NL-92

already band 11 05
- 79.142.76.101

In [12]:
# import random 
# nums = [random.randint(7, 21) for i in range(15)]
# stops = [sum(nums[:i+1])  for i in range(len(nums))]
# print(nums)
# print(stops)
# print(random.choice(nums))

In [13]:
def for_next_request(iter, userag):
    # , userag
    must_stop = False
    agent = UserAgent().chrome
    while agent == userag:
        agent = UserAgent().chrome
    nums = [20, 15, 19, 18, 16, 18, 15, 7, 12, 18, 15, 19, 18, 18, 14, 12]
    stops = [7, 15, 34, 52, 68, 86, 101, 108, 120, 138, 153, 172, 190, 208, 222, 234]
    # nums = [25, 18, 11, 13, 13, 10, 20, 14]
    # stops = [7, 18, 29, 42, 55, 65, 83, 97]
    if iter >= max(stops):
        must_stop = True        
        return must_stop, userag 
    if iter in stops:
        ts = int(random.choice(nums))
        print(f"ts = {ts}")
        time.sleep(ts)
    return must_stop, agent

In [14]:
# testing 
# output = "write"
# cur_web = 'offline'
# saveingDB = 'easy'
# save_html_status = "saving"

# -------------
# working small group of authors 
# output = "write"
# cur_web = 'online'
# saveingDB = 'write'
# save_html_status = "saving"

# -________________
# scraping
output = "not_write"
cur_web = 'online'
saveingDB = 'easy'
save_html_status = "not_saving"
last_au = {'name': 'Галактионов Владимир Александрович', 'mn_id': '62322'}
# {'name': 'Волобой Алексей Геннадьевич', 'mn_id': '79673'}
# {'name': 'Волков Юрий Александрович', 'mn_id': '27366'}
#{'name': 'Вабищевич Петр Николаевич', 'mn_id': '28431'}
# {'name': 'Быковская Елена Николаевна', 'mn_id': '165238'}
#{'name': 'Брушлинский Константин Владимирович', 'mn_id': '20345'}
# {'name': 'Борисов Леонид Андреевич', 'mn_id': '82515'}
# {'name': 'Бойко Анна Ивановна', 'mn_id': '145414'}
# {'name': 'Богданов Илья Петрович', 'mn_id': '137771'}
# {'name': 'Белов И В', 'mn_id': '33099'}
# {'name': 'Батищева Янина Генриховна', 'mn_id': '31576'}
# {'name': 'Багиров Лев Аркадьевич', 'mn_id': '19296'}
# {'name': 'Бабенко Константин Иванович', 'mn_id': '21883'}

iter = 1
must_stop = False
userag = UserAgent().chrome
for indx, item in authors_dict.items():
    random.seed(3017)
    if need_to_scrap(item, audb, inlist = False) or (item['mn_id'] == last_au['mn_id']):
        print(f"-------- iter = {iter}")
    # if need_to_scrap(item, audb, inlist = True):                        
        new_author = Author(mn_id=item["mn_id"])
        print(item)
        # get through pubs list on author page  
        r = random.random()
        
        must_stop, userag = for_next_request(iter, userag)
        if must_stop: 
            print(item)           
            break
        author_pubs = get_author_info(item["mn_id"], status=save_html_status,web=cur_web,rand = r,agent = userag)
        iter += 1
        
        if author_pubs is None:
            continue
        for pubs_indx, pubs_item in author_pubs.items():
            print(f"-------- iter = {iter}")
            if 'mn_link' not in pubs_item:
                continue
            if output == "write":                
                print(pubs_indx, pubs_item)            
            if pubs_item["mn_link"] in pubdb.db.keys():
                print(f'we have {pubs_item["mn_link"]} in pubdb.db.keys()')
                # статья уже есть надо просто добавить к автору
                new_author.read_from_pubdb(pubs_item["mn_link"], audb)                
                continue
            
            if pubs_item["mn_link"] is not None:
                r = random.random()
                must_stop, userag = for_next_request(iter, userag)
                if must_stop:
                    print(item)            
                    break                                                
                pub_page_info = get_pub_info(item["mn_id"],pubs_item["mn_link"],status=save_html_status,web=cur_web,rand = r,agent = userag)
                iter += 1
                new_pub = Publication(pubs_item["mn_link"])
                new_abs = Abstract(pubs_item["mn_link"])
                
                if pub_page_info is not None:                    
                    new_author.update_author_info(pubs_item, pub_page_info)
                    new_pub.update_publication_info(pubs_item, pub_page_info)
                    new_abs.update_abstract_info(pubs_item, pub_page_info)                    
                    pubdb.update_data(new_pub.convert2dict())                            
                    absdb.update_data(new_abs.convert2dict())
                    if output == "write":
                        for page_indx, page_item  in pub_page_info.items():
                            print(page_indx)
                            print(page_item)                              
        print(new_author.convert2dict())
        audb.update_data(new_author.convert2dict())
        if output  == "write":
            audb.show()
            pubdb.show()
            absdb.show()
        audb.save(status = saveingDB,text = f"we saving after {item['mn_id']} author")
        pubdb.save(status = saveingDB)
        absdb.save(status = saveingDB)
        
        print(f"we end to read {item}")    
        time.sleep(30*(2+random.random()))

audb.save(status = 'write',text = f"we saving after FULL SEARCH :-)")
pubdb.save(status = 'write')
absdb.save(status = 'write')        

We don't have him {'name': 'Волобой Алексей Геннадьевич', 'mn_id': '79673'} in authors_db
-------- iter = 1
{'name': 'Волобой Алексей Геннадьевич', 'mn_id': '79673'}
../data/79673/mn_author_79673_page.pkl
False
we need to request page for author mnid = 79673
-------- iter = 2
-------- iter = 2
-------- iter = 2
we have ipmp3006 in pubdb.db.keys()
For author 27366 cant find this ipmp3006 in Author
-------- iter = 2
we have tisp583 in pubdb.db.keys()
For author 27366 cant find this tisp583 in Author
-------- iter = 2
we have ipmp2898 in pubdb.db.keys()
-------- iter = 2
we have ipmp2872 in pubdb.db.keys()
For author 27366 cant find this ipmp2872 in Author
-------- iter = 2
we have ipmp2871 in pubdb.db.keys()
For author 27366 cant find this ipmp2871 in Author
-------- iter = 2
we have tisp486 in pubdb.db.keys()
-------- iter = 2
we have tisp485 in pubdb.db.keys()
-------- iter = 2
../data/79673/ipmp2705.pkl
False
we don't find that file
we need to request page for pub_page mnlink = ipmp27

In [15]:
audb.save(status = 'write',text = f"we saving after fail to do full search)")
pubdb.save(status = 'write')
absdb.save(status = 'write')

we saving after fail to do full search)
